This code is incomplete. Your job is to finish the sections which are clearly marked with all caps.

## Tool Calling

Function calling lets a language model go beyond just generating text — it can decide when to use specific tools or functions based on what the user says. For example, if someone asks “what’s 3 times 5?”, the model can respond with a structured output like {"name": "multiply", "arguments": {"x": 3, "y": 5}}, which your program can then use to run the actual multiply function. This works by giving the model a clear prompt that explains what tools are available and how to respond in a specific format, usually JSON. With the right setup, even models that don’t have built-in tool support can still follow the instructions and act like smart assistants that trigger real code.

## Functions

In this tutorial, we’ll be working with functions and using function calling to make language models more useful and interactive. Function calling is especially powerful when the model needs to handle tasks it can’t do on its own — like checking the current weather or time, accessing private or proprietary data, or querying an external system like a SQL database. By setting up a system where the model can “ask” for a specific tool to be used, we can bridge the gap between the LLM and real-world functionality it wouldn’t otherwise have access to.

Lets take a look at some functions that an LLM would not naturally be able to get:

In [ ]:
from datetime import datetime
import json

# Function to get the current time
def get_current_time():
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return json.dumps({"current_time": current_time})

get_current_time()

In [ ]:
from weather import get_weather
get_weather("San Francisco")

In [ ]:
import sqlite3

# Create an in-memory SQLite database and populate it with sample data
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create a sample table and insert some data
cursor.execute('''
    CREATE TABLE employees (
        id INTEGER PRIMARY KEY,
        name TEXT,
        position TEXT,
        salary INTEGER
    )
''')

sample_data = [
    (1, 'Alice', 'Engineer', 75000),
    (2, 'Bob', 'Manager', 90000),
    (3, 'Charlie', 'Analyst', 60000),
    (4, 'Diana', 'HR', 50000)
]

cursor.executemany('INSERT INTO employees VALUES (?, ?, ?, ?)', sample_data)
conn.commit()

# Define the function to execute SQL queries
def execute_sql_query(query: str):
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        return json.dumps({"results": results})
    except Exception as e:
        return json.dumps({"error": str(e)})

In [ ]:
execute_sql_query("SELECT * FROM employees")

## Describe Function

Before an LLM can use a function, we need to describe that function in a way the model can understand. This is usually done using a JSON-like dictionary that includes:

- The function’s name

- A description of what it does

- Its parameters, including types and optional descriptions for each argument

You can write these descriptions manually, or even ask an LLM to help generate them — and then review or modify the result as needed.

Let’s create three example tools: one to fetch the weather for a given city, another to get the current system time, and another to run a SQL query on our database. Fill in the TODOs.

In [ ]:
weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Fetches the current weather for a given city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The name of the city."
                }
            },
            "required": ["city"]
        }
    }
}


time_tool = {
    "type": "function",
    "function": {
        "name": "get_current_time",
        "description": "ENTER DESCRIPTION FOR TIME TOOL HERE.",
        "parameters": {}
    }
}

sql_tool = {
    "type": "function",
    "function": {
        "name": "execute_sql_query",
        "description": "Execute an SQL query on database with table name 'employees' and return the results. The schema is id, name, position, salary",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The SQL query to execute."
                }
            },
            "required": ["query"]
        }
    }
}

## Ollama Setup

In [ ]:
import os
from ollama import Client

# Read the port from the file
with open(os.path.expanduser('~/.ollama_port')) as f:
    port = f.read().strip()

# Connect to 127.0.0.1:<port>
host = f"http://127.0.0.1:{port}"

client = Client(host=host)

In [ ]:
# Get LLM
client.pull("qwen3:4b")

## Tool Calling Setup

Define the tool functions available to the ollama model.

In [ ]:
TOOL_FUNCTIONS = {
    "get_weather": get_weather,
    "get_current_time": get_current_time,
    "execute_sql_query": execute_sql_query
}

System prompts allow the programmer to achieve greater control over the LLM's use of tools and higher-level context. 

They are not visible to the user but instead are a distinct section of the LLM's context window. System prompts often include phrases like "Use tools only when necessary." or give the LLM useful meta-context such as "You are a helpful assistant."

Here you can experiment with the system prompt and understand how it affects the later queries.

In [ ]:
# TODO
system_prompt = "ENTER YOUR SYSTEM PROMPT HERE."

In [ ]:
import ollama

def chat(prompt):
    response = client.chat(
        model='qwen3:4b',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        tools=[weather_tool, time_tool, sql_tool],
        options={
            'temperature': 0
        }
    )

    if "tool_calls" in response["message"]:
        for tool_call in response["message"]["tool_calls"]:
            tool_name = tool_call["function"]["name"]
            args = tool_call["function"].get("arguments", {})
            print(f"Tool: {tool_name}")

            # Ensure the tool exists
            if tool_name not in TOOL_FUNCTIONS:
                print(f"Tool '{tool_name}' is not implemented.")
                continue

            try:
                # Call the actual tool function
                tool_output = json.loads(TOOL_FUNCTIONS[tool_name](**args))
            except Exception as e:
                tool_output = {"error": str(e)}

            if "error" in tool_output:
                final_response = tool_output["error"]
            else:
                followup = client.chat(
                    model='qwen3:4b',
                    messages=[
                        {"role": "system", "content": f"You are a helpful assistant. Turn tool outputs into natural conversational replies. If weather tool was used, please convert celsius to fahrenheit."},
                        {"role": "user", "content": f"Here is the output of the tool '{tool_name}': {tool_output}, with the prompt: {prompt}. Reply as an answerer."}
                    ]
                )
                final_response = followup["message"]["content"]

            print(final_response)
    else:
        print(response["message"]["content"])


## Using the Tool-calling LLM

- Ask some time related questions (May take some time to complete)

In [ ]:
# TODO
chat("ENTER YOUR PROMPT HERE.")

- Ask what the weather is like in whichever city you want to ask about.

In [ ]:
# TODO
chat("ENTER YOUR PROMPT HERE.")

- Ask about the company that we had put in the SQL database

In [ ]:
# TODO
chat("ENTER YOUR PROMPT HERE.")

- Run a prompt that is unrelated to any of the tools, and see what the LLM does.

In [ ]:
# TODO
chat("Foo Bar Foo Bar")